РАБОТАЕМ С IGNITE С ПОМОЩЬЮ БИБЛИОТЕКИ PYIGNITE

Данильченко Вадим

In [2]:
# установим нужные библиотеки
!pip install folium==0.2.1
!pip install pyignite==0.2.0

In [1]:
# импортируем библиотеки
from pyignite import Client
import re
from tqdm import tqdm

In [2]:
# установим подключение
client = Client()
client.connect('0.0.0.0', 32773)

In [3]:
# откроем файл с запросом
with open('./world.sql', 'r') as f:
    sql = f.read().splitlines()
    
sql[:10]

['DROP TABLE IF EXISTS Country;',
 '',
 'CREATE TABLE Country (',
 '  Code CHAR(3) PRIMARY KEY,',
 '  Name VARCHAR,',
 '  Continent VARCHAR,',
 '  Region VARCHAR,',
 '  SurfaceArea DECIMAL(10,2),',
 '  IndepYear SMALLINT,',
 '  Population INT,']

-----------------------------------------------------------------------------------------------------------------------------
создадим кэш для Country


In [4]:
# создадаим кэш
cache_country = client.create_cache('Country')

In [5]:
# запишем в него данные, где ключом будет название страны, а значением ее код
for line in tqdm(sql):
    if "INSERT INTO Country(" in line:
        v, k = [i.strip() for i in re.sub("[^a-zA-Z\ \-\,]", "", line.split('VALUES')[-1]).split(',')[:2]]
        cache_country.put(k, v)

100%|████████████████████████████████████| 5354/5354 [00:00<00:00, 10490.07it/s]


In [6]:
# выведем все значения нашей таблицы
result = cache_country.scan()

for k, v in result:
    print(k, v)

Cambodia KHM
Paraguay PRY
Syria SYR
Solomon Islands SLB
Montserrat MSR
Guadeloupe GLP
Falkland Islands FLK
Seychelles SYC
Bahrain BHR
Comoros COM
Faroe Islands FRO
Finland FIN
Puerto Rico PRI
Eritrea ERI
Liechtenstein LIE
Jordan JOR
Bulgaria BGR
United Arab Emirates ARE
Kenya KEN
French Polynesia PYF
Djibouti DJI
Cuba CUB
Saint Lucia LCA
Mayotte MYT
Israel ISR
San Marino SMR
Tajikistan TJK
Gibraltar GIB
Cyprus CYP
Northern Mariana Islands MNP
Malaysia MYS
Armenia ARM
Brazil BRA
Turks and Caicos Islands TCA
Holy See Vatican City State VAT
Ecuador ECU
United States Minor Outlying Islands UMI
Italy ITA
Haiti HTI
Afghanistan AFG
Russian Federation RUS
American Samoa ASM
Kyrgyzstan KGZ
Togo TGO
Cocos Keeling Islands CCK
Uzbekistan UZB
Dominica DMA
Benin BEN
East Timor TMP
Sudan SDN
Portugal PRT
Grenada GRD
Latvia LVA
Mongolia MNG
Morocco MAR
Guatemala GTM
Heard Island and McDonald Islands HMD
Tanzania TZA
Ghana GHA
India IND
Canada CAN
Maldives MDV
Taiwan TWN
Central African Republic CAF
Ge

In [9]:
# найдем несколько значений по ключу
cache_country.get_all(['Micronesia', 'Niger', 'Sri Lanka'])

{'Niger': 'NER', 'Sri Lanka': 'LKA', 'Micronesia': 'FSM'}

-----------------------------------------------------------------------------------------------------------------------------
создадим кэш для City


In [27]:
# создадаим кэш
cache_city = client.create_cache('City')

In [28]:
# запишем в него данные, где ключом будет id-города, а значением код страны
for line in tqdm(sql):
    if "INSERT INTO City(" in line:
        k, v = [j.strip() for i,j in enumerate(re.sub("[^a-zA-Z0-9\ \-\,]", "", 
                                                      line.split('VALUES')[-1]).split(',')) if i in [0,2]]
        cache_city.put(k, v)

100%|█████████████████████████████████████| 5354/5354 [00:04<00:00, 1245.39it/s]


In [29]:
# выведем все значения нашей таблицы
result = cache_city.scan()

i = 0
for k, v in result:
    print(k, v)
    i+=1
    if i>10:
        break

907 GEO
908 GEO
909 GEO
3880 USA
3760 RUS
3881 USA
3640 RUS
3761 RUS
3882 USA
1220 IND
1341 IND


In [31]:
# найдем несколько значений по ключу
cache_city.get_all(['1', '128', '333'])

{'1': 'AFG', '128': 'ARM', '333': 'BRA'}

-----------------------------------------------------------------------------------------------------------------------------
создадим кэш для CountryLanguage


In [32]:
# создадаим кэш
cache_country_language = client.create_cache('CountryLanguage')

In [33]:
# запишем в него данные, где ключом будет название страны, а значением ее код
for line in tqdm(sql):
    if "INSERT INTO CountryLanguage(" in line:
        k, v = [i.strip() for i in re.sub("[^a-zA-Z\ \-\,]", "", line.split('VALUES')[-1]).split(',')[:2]]
        cache_country_language.put(k, v)

100%|█████████████████████████████████████| 5354/5354 [00:00<00:00, 6112.43it/s]


In [34]:
# выведем все значения нашей таблицы
result = cache_country_language.scan()

for k, v in result:
    print(k, v)

NZL Maori
FJI Hindi
GLP French
WLF Wallis
SDN Zande
GMB Wolof
MYS Tamil
MYT Malagasy
GNB Portuguese
SEN Wolof
TGO Watyi
IRL Irish
FLK English
IRN Turkmenian
QAT Urdu
BDI Swahili
IRQ Persian
GNQ Fang
THA Thai
ABW Spanish
ISL Icelandic
BEL Turkish
ISR Russian
KWT English
BEN Somba
ITA Slovene
PAK Urdu
BFA Mossi
PAN Spanish
SGP Tamil
UKR Ukrainian
CHE Romansh
DJI Somali
REU Tamil
CHL Spanish
CHN Zhuang
HRV Slovene
TJK Uzbek
VNM Vietnamese
SHN English
BGD Tripuri
ZWE Shona
YUG Slovak
BGR Turkish
UMI English
CIV SouthMande
TKL Tokelau
TKM Uzbek
GRD Creole English
GRC Turkish
PCN Pitcairnese
HTI Haiti Creole
GRL Greenlandic
AFG Uzbek
BHS Creole French
BHR English
SJM Russian
DMA Creole French
BIH Serbo-Croatian
HUN Slovak
AGO Ovimbundu
WSM Samoan-English
FRA Turkish
TMP Sunda
NAM San
PER Spanish
DNK Turkish
GTM Spanish
FRO Faroese
SLB Polynesian Languages
SLE Yalunka
AIA English
GUF Indian Languages
SLV Spanish
GUM Philippene Languages
FSM Yap
DOM Spanish
CMR Tikar
GUY Creole English
MAC Por

In [37]:
# найдем несколько значений по ключу
cache_country_language.get_all(['NER', 'LKA', 'FSM', 'RUS'])

{'FSM': 'Yap', 'RUS': 'Ukrainian', 'NER': 'Tamashek', 'LKA': 'Tamil'}

результат объясняется тем, что записывалась последняя строка для страны, соответственно при выводе может отображаться не самый популярный язык

### SQL. IGNITE

In [55]:
# Пропишем код для создания трех таблиц Country, City и CountryLanguage:
COUNTRY_CREATE_TABLE_QUERY = '''CREATE TABLE Country (
    Code CHAR(3) PRIMARY KEY,
    Name CHAR(52),
    Continent CHAR(50),
    Region CHAR(26),
    SurfaceArea DECIMAL(10,2),
    IndepYear SMALLINT(6),
    Population INT(11),
    LifeExpectancy DECIMAL(3,1),
    GNP DECIMAL(10,2),
    GNPOld DECIMAL(10,2),
    LocalName CHAR(45),
    GovernmentForm CHAR(45),
    HeadOfState CHAR(60),
    Capital INT(11),
    Code2 CHAR(2)
)'''

CITY_CREATE_TABLE_QUERY = '''CREATE TABLE City (
    ID INT(11),
    Name CHAR(35),
    CountryCode CHAR(3),
    District CHAR(20),
    Population INT(11),
    PRIMARY KEY (ID, CountryCode)
) WITH "affinityKey=CountryCode"'''

LANGUAGE_CREATE_TABLE_QUERY = '''CREATE TABLE CountryLanguage (
    CountryCode CHAR(3),
    Language CHAR(30),
    IsOfficial BOOLEAN,
    Percentage DECIMAL(4,1),
    PRIMARY KEY (CountryCode, Language)
) WITH "affinityKey=CountryCode"'''

In [56]:
# исполним код
for query in [
    COUNTRY_CREATE_TABLE_QUERY,
    CITY_CREATE_TABLE_QUERY,
    LANGUAGE_CREATE_TABLE_QUERY,
]:
    client.sql(query)

In [57]:
# Создадим индексы для таблиц City и CountryLanguage
CITY_CREATE_INDEX = '''
CREATE INDEX idx_country_code ON city (CountryCode)'''

LANGUAGE_CREATE_INDEX = '''
CREATE INDEX idx_lang_country_code ON CountryLanguage (CountryCode)'''

for query in [CITY_CREATE_INDEX, LANGUAGE_CREATE_INDEX]:
    client.sql(query)

In [60]:
# Добавим данных в таблицы:
city = ["INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (1,'Kabul','AFG','Kabol',1780000);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (2,'Qandahar','AFG','Qandahar',237500);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (3,'Herat','AFG','Herat',186800);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (4,'Mazar-e-Sharif','AFG','Balkh',127800);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (5,'Amsterdam','NLD','Noord-Holland',731200);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (6,'Rotterdam','NLD','Zuid-Holland',593321);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (7,'Haag','NLD','Zuid-Holland',440900);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (8,'Utrecht','NLD','Utrecht',234323);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (9,'Eindhoven','NLD','Noord-Brabant',201843);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (10,'Tilburg','NLD','Noord-Brabant',193238);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (11,'Groningen','NLD','Groningen',172701);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (12,'Breda','NLD','Noord-Brabant',160398);",
"INSERT INTO City(ID, Name, CountryCode, District, Population) VALUES (13,'Apeldoorn','NLD','Gelderland',153491);"
        
]


for row in city:
    client.sql(row)

country = ["INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ABW','Aruba','North America','Caribbean',193.00,NULL,103000,78.4,828.00,793.00,'Aruba','Nonmetropolitan Territory of The Netherlands','Beatrix',129,'AW');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('AFG','Afghanistan','Asia','Southern and Central Asia',652090.00,1919,22720000,45.9,5976.00,NULL,'Afganistan/Afqanestan','Islamic Emirate','Mohammad Omar',1,'AF');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('AGO','Angola','Africa','Central Africa',1246700.00,1975,12878000,38.3,6648.00,7984.00,'Angola','Republic','José Eduardo dos Santos',56,'AO');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('AIA','Anguilla','North America','Caribbean',96.00,NULL,8000,76.1,63.20,NULL,'Anguilla','Dependent Territory of the UK','Elisabeth II',62,'AI');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ALB','Albania','Europe','Southern Europe',28748.00,1912,3401200,71.6,3205.00,2500.00,'Shqipëria','Republic','Rexhep Mejdani',34,'AL');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('AND','Andorra','Europe','Southern Europe',468.00,1278,78000,83.5,1630.00,NULL,'Andorra','Parliamentary Coprincipality','',55,'AD');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ANT','Netherlands Antilles','North America','Caribbean',800.00,NULL,217000,74.7,1941.00,NULL,'Nederlandse Antillen','Nonmetropolitan Territory of The Netherlands','Beatrix',33,'AN');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ARE','United Arab Emirates','Asia','Middle East',83600.00,1971,2441000,74.1,37966.00,36846.00,'Al-Imarat al-´Arabiya al-Muttahida','Emirate Federation','Zayid bin Sultan al-Nahayan',65,'AE');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ARG','Argentina','South America','South America',2780400.00,1816,37032000,75.1,340238.00,323310.00,'Argentina','Federal Republic','Fernando de la Rúa',69,'AR');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ARM','Armenia','Asia','Middle East',29800.00,1991,3520000,66.4,1813.00,1627.00,'Hajastan','Republic','Robert Kotšarjan',126,'AM');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ASM','American Samoa','Oceania','Polynesia',199.00,NULL,68000,75.1,334.00,NULL,'Amerika Samoa','US Territory','George W. Bush',54,'AS');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ATA','Antarctica','Antarctica','Antarctica',13120000.00,NULL,0,NULL,0.00,NULL,'–','Co-administrated','',NULL,'AQ');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ATF','French Southern territories','Antarctica','Antarctica',7780.00,NULL,0,NULL,0.00,NULL,'Terres australes françaises','Nonmetropolitan Territory of France','Jacques Chirac',NULL,'TF');",
"INSERT INTO Country(Code, Name, Continent, Region, SurfaceArea, IndepYear, Population, LifeExpectancy, GNP, GNPOld, LocalName, GovernmentForm, HeadOfState, Capital, Code2) VALUES ('ATG','Antigua and Barbuda','North America','Caribbean',442.00,1981,68000,70.5,612.00,584.00,'Antigua and Barbuda','Constitutional Monarchy','Elisabeth II',63,'AG');"
      
]

for row in country:
    client.sql(row)

clang = ["INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ABW','Dutch','T',5.3);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ABW','English','F',9.5);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ABW','Papiamento','F',76.7);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ABW','Spanish','F',7.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AFG','Balochi','F',0.9);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AFG','Dari','T',32.1);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AFG','Pashto','T',52.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AFG','Turkmenian','F',1.9);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AFG','Uzbek','F',8.8);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Ambo','F',2.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Chokwe','F',4.2);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Kongo','F',13.2);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Luchazi','F',2.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Luimbe-nganguela','F',5.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Luvale','F',3.6);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Mbundu','F',21.6);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Nyaneka-nkhumbi','F',5.4);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AGO','Ovimbundu','F',37.2);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AIA','English','T',0.0);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ALB','Albaniana','T',97.9);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ALB','Greek','F',1.8);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('ALB','Macedonian','F',0.1);",
"INSERT INTO CountryLanguage(CountryCode, Language, IsOfficial, Percentage) VALUES ('AND','Catalan','T',32.3);"
         

]


for row in clang:
    client.sql(row)

напишем несколько запросов к таблицам

In [61]:
query = '''
SELECT name, population FROM City ORDER BY population DESC LIMIT 10'''

result = client.sql(query)

print('Most 10 populated cities:')
for row in result:
    print(row)

Most 10 populated cities:
['Kabul', 1780000]
['Amsterdam', 731200]
['Rotterdam', 593321]
['Haag', 440900]
['Qandahar', 237500]
['Utrecht', 234323]
['Eindhoven', 201843]
['Tilburg', 193238]
['Herat', 186800]
['Groningen', 172701]


In [63]:
query = '''
SELECT name, population FROM City where population>1000000 ORDER BY population DESC LIMIT 10 '''

result = client.sql(query)

print('Top 10 cities with population over 1 bln:')
for row in result:
    print(row)

Top 10 cities with population over 1 bln:
['Kabul', 1780000]


In [68]:
query = '''
SELECT name, population FROM City where population<200000 ORDER BY population DESC LIMIT 10 '''

result = client.sql(query)

print('Top 10 cities with population less than 200000 :')
for row in result:
    print(row)

Top 10 cities with population less than 200000 :
['Tilburg', 193238]
['Herat', 186800]
['Groningen', 172701]
['Breda', 160398]
['Apeldoorn', 153491]
['Mazar-e-Sharif', 127800]


In [90]:
query = '''
SELECT name, population FROM City where SUBSTRING(name, 1, 1) = 'a' '''

result = client.sql(query)

print('Top 10 cities with "a" on 1st place of city name :')
for row in result:
    print(row)

Top 10 cities with "a" on 1st place of city name :


In [89]:
query = '''
SELECT name, population FROM City where SUBSTRING(name, 2, 1) = 'a' '''

result = client.sql(query)

print('Top 10 cities with "a" on 2nd place of city name :')
for row in result:
    print(row)

Top 10 cities with "a" on 2nd place of city name :
['Kabul', 1780000]
['Qandahar', 237500]
['Mazar-e-Sharif', 127800]
['Haag', 440900]


In [94]:
query = '''
SELECT Continent, count(Name) cnt_countries FROM Country group by Continent'''

result = client.sql(query)

print('count countries by Continent :')
for row in result:
    print(row)

count countries by Continent :
['Asia', 3]
['North America', 4]
['South America', 1]
['Africa', 1]
['Oceania', 1]
['Antarctica', 2]
['Europe', 2]
